In [103]:
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.sppQ73koN2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.B7qpUEIjsY/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.xk9dMt0ycB/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [104]:
import time
import pandas as pd
import numpy as np
import time
from datetime import datetime
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import requests

In [77]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [200]:
url = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN0300&steadyYn=Y'
driver.get(url)
req = driver.get(url)

##**제목 크롤링**

In [201]:
title = driver.find_elements(By.CLASS_NAME,'ellipsis.rank01')
title_use = []
for i in title:
  title_use.append(i.text)

##**가사 크롤링**

In [202]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req = requests.get('https://www.melon.com/genre/song_list.htm?gnrCode=GN0300&steadyYn=Y', headers = header) ## 주간 차트를 크롤링 할 것임
html = req.text
parse = bs(html,'html.parser')

lyrics = parse.find_all("div",{"class": "ellipsis rank01"})

lyric = []

for i in lyrics :
  lyric.append(i.find('a'))

pattern = re.compile(r"javascript:melon.play.playSong\('\d+',(\d+)\);")

# 각 <a> 요소를 추출하여 텍스트로 변환하고 lyric 리스트에 추가
for i in lyrics:
    a_tag = i.find('a')
    if a_tag:
        href = a_tag['href']
        match = pattern.search(href)

        if match:
            number = match.group(1)
            lyric.append(number)
for number in lyric:
    print(number)
number = []
number = lyric[50:100]

<a href="javascript:melon.play.playSong('25240105',531840);" title="Y (Please Tell Me Why) 재생">Y (Please Tell Me Why)</a>
<a href="javascript:melon.play.playSong('25240105',1850253);" title="우산 (Feat. 윤하) 재생">우산 (Feat. 윤하)</a>
<a href="javascript:melon.play.playSong('25240105',5650996);" title="우연히 봄 재생">우연히 봄</a>
<a href="javascript:melon.play.playSong('25240105',3853980);" title="어땠을까 (Feat. 박정현) 재생">어땠을까 (Feat. 박정현)</a>
<a href="javascript:melon.play.playSong('25240105',2312253);" title="헤어지지 못하는 여자, 떠나가지 못하는 남자 (Feat. 정인) 재생">헤어지지 못하는 여자, 떠나가지 못하는 남자 (Feat. 정인)</a>
<a href="javascript:melon.play.playSong('25240105',4681146);" title="한여름밤의 꿀 재생">한여름밤의 꿀</a>
<a href="javascript:melon.play.playSong('25240105',845586);" title="I Love U Oh Thank U (Feat. 김태우 Of God) 재생">I Love U Oh Thank U (Feat. 김태우 Of God)</a>
<a href="javascript:melon.play.playSong('25240105',522852);" title="너에게 쓰는 편지 (Feat. 린) 재생">너에게 쓰는 편지 (Feat. 린)</a>
<a href="javascript:melon.play.playSong('25240105',3726617);"

In [203]:
print(len(number))
print(number[0],number[49])

50
531840 4226211


In [207]:
nl = []

for num in number:
    driver.get("https://www.melon.com/song/detail.htm?songId=" + num)
    ne = driver.find_element(By.CLASS_NAME, value="lyric")
    nl.append(ne.text)


clear = []

for text in nl:
    clear.append(text.replace("\n", ""))

NoSuchElementException: ignored

In [ ]:
for i in clear:
  print(i)

In [197]:
print(len(number))
print(len(clear))
print(len(nl))

50
50
50


##**input data 형식을 위한 장르 추가**

In [198]:
genre = []
use_genre = "트로트"
for i in range(50):
  genre.append(use_genre)

##**Data Frame > CSV 변환**

In [199]:
df = pd.DataFrame({"장르":genre,"제목":title_use,"가사":clear})
df.to_csv("Trott_use.csv", encoding="utf-8")